In [3]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [1]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})


In [4]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.enjoytokyo.jp/event/list/area1306/',
   'title': "東京駅周辺・丸の内でおすすめのイベント - Let's ENJOY TOKYO",
   'content': '～ 2025/06/01(日) 休館日：月曜日（5月5日は開館）、5月7日 ～ 2025/06/01(日) ～ 2025/06/01(日) 休館日：月曜日（5月5日は開館）、5月7日 ～ 2025/06/03(火) ～ 2025/06/03(火) ～ 2025/06/03(火) 2025/05/29(木) ～ 09/07(日) 伊勢丹 立川店：3/26〜4/15 国際フォーラム：7/14〜7/31 日本橋三越本店：8/6〜8/19 文京グリーンコート：9/1〜9/30 2025/05/21(水) ～ 06/30(月) 熊本が誇る旬の逸品“春スイカ”を首都圏で楽しめる「熊本 春スイカ フェスタ」が開催！ 2025/04/08(火) ～ 07/27(日) 休室日：月曜日、7月8日(火)～13日(日) 2025/05/19(月) ～ 06/08(日) ～ 2025/05/31(土) 2025/06/21(土) ～ 06/22(日) 2025/06/21(土) ～ 06/22(日) 2026/02/07(土) ～ 05/24(日) 休館日：2026/02/16(月)、03/16(月)、04/13(月)、05/11(月) 20世紀北欧デザインの巨匠 スティグ・リンドベリ展 2025/08/20(水) ～ 09/07(日) 20世紀北欧デザインの巨匠 スティグ・リンドベリ展 20世紀北欧デザインの巨匠 スティグ・リンドベリの作品約300点を展示！ 2025/07/26(土) ～ 08/15(金) 2025/06/06(金) ～ 07/06(日) 2025/06/14(土) ～ 07/21(月・祝) 2025/05/31(土) ～ 06/01(日) ～ 2025/05/31(土) 2025/05/13(火) ～ 07/06(日) 休館日は月曜日。開映日時は作品によって異なる。 天空茶会第二十四葉 特別セミナー世界の茶産地を訪ねて セミナー午前の部\u3000ネパールとの出会い 天空茶会第二十四葉 特別セミ

In [5]:
# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [6]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [7]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [8]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [9]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- 東京都の面積は約2,194平方キロメートルです。
- 沖縄県の面積は約2,271平方キロメートルです。

したがって、広さで言えば沖縄県が東京都よりも広いです。


In [10]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
最近1ヶ月以内の東京駅でのイベント情報は以下の通りです：

1. **[東京駅周辺のイベント一覧 - マイナビニュース](https://www.enjoytokyo.jp/event/list/)**
   - **日付**: 2025年6月1日（予定）
   - **内容**: 「TOKYO COFFEE FESTIVAL 2025 Spring」、「東京アウトレットウィーク」、「xikers 2025 WORLD TOUR」などが予定されています。

2. **[東京駅周辺のイベント - Walkerplus](https://www.walkerplus.com/event_list/ar0313/sc309880d/)**
   - **日付**: 2025年5月29日から2025年6月7日
   - **内容**: 「女優の早川友里亜によるクリエイター同好会とのコラボイベント」が開催されます。

3. **[屋内で楽しむイベント情報 - 楽天トラベル](https://travel.rakuten.co.jp/mytrip/ranking/indoor-spot-tokyo)**
   - **内容**: 雨の日や暑い日でも楽しめる屋内イベントとしていくつかの施設が紹介されています。

4. **[新宿のイベント情報](https://6museums.tokyo/)**
   - **内容**: 新宿周辺の美術館でも常設展示が行われており、東京駅からのアクセスが良好です。

5. **[2025年7月までの新発表イベント - JR東日本](https://media.jreast.co.jp/articles/3405)**
   - **日付**: 2025年4月14日までのさまざまなイベント予定が発表されています。新しい路線や特別列車が期間限定で運行される予定です。

詳細やその他の情報は、各リンクを参照してください。


In [11]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:こんにちは'

こんにちは！今日はどんなことをお手伝いできますか？


'質問:東北6県は？'

東北地方は、以下の6つの都道府県で構成されています。

1. 青森県（あおもりけん）
2. 岩手県（いわてけん）
3. 宮城県（みやぎけん）
4. 秋田県（あきたけん）
5. 山形県（やまがたけん）
6. 福島県（ふくしまけん）

これらの県は日本の東北に位置し、自然の美しさや豊かな文化が特徴です。


'質問:宮城県のお土産について検索した結果を教えて'

宮城県のお土産についての情報を以下にまとめました。

1. **お土産の種類**
   - 宮城県は、牛たんやずんだ餅といった名物が有名です。また、地元の特産品やスイーツ、飲料も多くあります。

2. **おすすめのお土産**
   - **牛たん**: 宮城県を代表する名物。焼き方や味付けにこだわった商品が多彩です。
   - **ずんだ餅**: 枝豆を使った餅で、甘さと食感が特徴的です。
   - **萩の月**: 宮城の洋菓子で、カスタードクリームを包んだスポンジケーキです。
   - **仙台味噌**: 地元産の味噌も人気があり、家庭用や贈答用に喜ばれます。

3. **販売情報**
   - **量販店や観光地の店舗**: 宮城県内の多くの店舗でお土産を購入できます。仙台駅構内の土産物店も便利です。
   - **オンラインショップ**: 一部の名産品は、オンラインで購入することも可能です。

4. **関連リンク**
   - [宮城県のお土産の情報まとめ](https://atlas-log.com/miyagi-souvenir/)
   - [人気のスイーツや特産品のリスト](https://travel.rakuten.co.jp/mytrip/howto/sendai-souvenir)
   - [お土産ランキング一覧](https://www.jalan.net/omiyage/040000/)

以上が宮城県の主なお土産の情報です。旅行の際の参考にしてください。

---ご利用ありがとうございました！---
